## Шаг 1. Подготовка окружения

In [ ]:
!pip install locust mlflow -qqq
!pip install cryptography==41.0.5 evidently pandas numpy scikit-learn seaborn matplotlib -qqq
!pip install deepchecks -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 k

## Шаг 2. Принятие архитектурных решений (ADR)

Здесь я создаю ADR-каталог и документ, описывающий две альтернативные архитектуры ML-сервиса:

1. **Монолитное приложение** (один контейнер FastAPI)
2. **Микросервисная архитектура** (разделение API/inference/monitoring)

В ADR-файле описано:
- какие альтернативы рассматривались,
- почему был выбран текущий вариант,
- какие компромиссы и последствия это решение имеет.



In [ ]:
!git clone https://github.com/npryce/adr-tools.git >/dev/null

Cloning into 'adr-tools'...
remote: Enumerating objects: 793, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 793 (delta 186), reused 172 (delta 172), pack-reused 568 (from 1)
Receiving objects: 100% (793/793), 126.76 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (448/448), done.


In [ ]:
import os
os.environ["PATH"] += ":/content/adr-tools/src"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!adr init /content/drive/MyDrive/adr_records

/content/drive/MyDrive/adr_records/0001-record-architecture-decisions.md


In [ ]:
!adr new "Почему мы выбрали такую архитектуру"

/content/drive/MyDrive/adr_records/0002-.md


```yaml
openapi: 3.0.0
info:
  title: ML Service API
  version: "1.0.0"
  description: |
    REST API для сервиса предсказаний и мониторинга качества данных.

servers:
  - url: http://localhost:8000

paths:
  /healthcheck:
    get:
      summary: Проверка статуса сервиса
      responses:
        "200":
          description: OK

  /predict:
    post:
      summary: Выполнить предсказание модели
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                x:
                  type: number
      responses:
        "200":
          description: Предсказание модели
          content:
            application/json:
              schema:
                type: object
                properties:
                  prediction:
                    type: number

  /drift:
    post:
      summary: Анализ дрейфа данных
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                ref:
                  type: array
                  items:
                    type: number
                current:
                  type: array
                  items:
                    type: number
      responses:
        "200":
          description: Результат полученный от Evidently
```


In [7]:
mkdir -p infrastructure/ansible/roles/ml_service/tasks

In [8]:
!mkdir -p service

## Шаг 3. Реализация ML-сервиса (FastAPI)

На этом шаге создаю REST-сервис, состоящий из 3 ключевых эндпоинтов:

- `/healthcheck` — проверка работоспособности сервиса
- `/predict` — простое предсказание (пример ML-инференса)
- `/drift` — анализ дрейфа данных с использованием Evidently

Подход — **API-first**:  
сначала был создан OpenAPI-документ, затем — реализация сервиса.

Что обеспечивает:
- воспроизводимость
- детерминированный интерфейс
- простоту нагрузочного тестирования (Locust)


In [10]:
%%writefile service/main.py
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

app = FastAPI()

# ---------- MODELS FOR REQUESTS ----------

class PredictRequest(BaseModel):
    x: float

class DriftRequest(BaseModel):
    ref: list
    current: list

# ---------- ENDPOINTS ----------

@app.get("/healthcheck")
def healthcheck():
    return {"status": "ok"}

@app.post("/predict")
def predict(req: PredictRequest):
    # simple mock model: multiply by 2
    prediction = req.x * 2
    return {"prediction": prediction}

@app.post("/drift")
def drift(req: DriftRequest):
    df_ref = pd.DataFrame({"x": req.ref})
    df_current = pd.DataFrame({"x": req.current})

    report = Report(metrics=[DataDriftPreset()])
    report.run(reference_data=df_ref, current_data=df_current)

    result = report.as_dict()
    return result


Writing service/main.py


## Запуск сервиса

Далее запускаю FastAPI-сервис локально в фоне (`nohup uvicorn ...`), чтобы иметь возможность выполнять запросы и проводить нагрузочное тестирование через Locust.


In [11]:
!pip install uvicorn fastapi -qqq
!uvicorn service.main:app --host 0.0.0.0 --port 8000 --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [24140] using WatchFiles
INFO:     Started server process [24142]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24142]
INFO:     Stopping reloader process [24140]


In [12]:
!nohup uvicorn service.main:app --host 0.0.0.0 --port 8000 > logs.txt 2>&1 &

In [13]:
import requests
requests.get("http://0.0.0.0:8000/healthcheck").json()

{'status': 'ok'}

## Шаг 4. Нагрузочное тестирование (Locust)

Создаю сценарий нагрузочного тестирования:

- Проверяю `/healthcheck`
- Проверяю `/predict`
- Отправляю 10 пользователей с ростом 1 RPS в секунду
- Ограничение по времени: 30 секунд
- Итоговый отчёт скачивается в `report.html`

Цель теста:
Оценить стабильность API под параллельной нагрузкой и убедиться, что сервис масштабируется линейно.


In [14]:
%%writefile locustfile.py
from locust import HttpUser, task, between

class LoadTestingUser(HttpUser):
    wait_time = between(1, 3)

    @task
    def health(self):
        self.client.get("/healthcheck")

    @task
    def predict(self):
        # простое тело запроса
        self.client.post("/predict", json={"x": 10})

Writing locustfile.py


In [15]:
!locust -f locustfile.py --headless -u 10 -r 1 --run-time 30s --host http://0.0.0.0:8000 --html report.html

[2025-11-29 23:08:51,323] 01b22a2dd31c/INFO/locust.main: Starting Locust 2.42.6
[2025-11-29 23:08:51,323] 01b22a2dd31c/INFO/locust.main: Run time limit set to 30 seconds
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2025-11-29 23:08:51,325] 01b22a2dd31c/INFO/locust.runners: Ramping to 10 users at a rate of 1.00 per second
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
GET      /healthcheck       1     0(0.00%) |      6       6       6      6 |    0.00        0.00
POST     /predict       1     0(0.00%) |     12      12      12     12 |    0.00        0.00
--

## Шаг 5. Анализ дрейфа данных (Evidently)

На этом этапе моделирую сценарий изменения распределения данных.

Использую:

- `baseline` — исторические данные
- `current` — текущие данные, которые содержит аномально искажённые значения

Evidently генерирует HTML-отчёт `drift_report.html`.

Этот отчёт является элементом ML-pipeline, требуемым по условию задания.


In [16]:
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

baseline = pd.DataFrame({"value": [1,2,3,4,5,6,7,8,9,10]})
current  = pd.DataFrame({"value": [2,3,4,50,60,70,3,4,5,6]})

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=baseline, current_data=current)
report.save_html("drift_report.html")

print("Drift report saved!")


Drift report saved!


## Шаг 6. MLflow — логирование параметров и метрик

MLflow используется как компонент ML-pipeline для:

- сохранения параметров запуска,
- сохранения метрик модели,
- фиксации экспериментов.

Это демонстрирует интеграцию воспроизводимой ML-части в DevOps-проект.


In [17]:
import mlflow

mlflow.set_experiment("demo-experiment")

with mlflow.start_run():
    mlflow.log_param("example_input", 10)
    mlflow.log_metric("demo_accuracy", 0.95)

print("MLflow logging complete!")

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning:

Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534

2025/11/29 23:56:03 INFO mlflow.tracking.fluent: Experiment with name 'demo-experiment' does not exist. Creating a new experiment.


MLflow logging complete!
